In [3]:
import cv2
import os
import sys
import time
import numpy as np 
import json
import skimage
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score


In [2]:
clases = os.listdir("raw-img")
print("CLASES:" , len(clases) , clases)

CLASES: 10 ['perro', 'caballo', 'mariposa', 'vaca', 'gallina', 'ardilla', 'elefante', 'araña', 'gato', 'oveja']


In [5]:
# Creamos las listas donde almacenaremos las características y las etiquetas
X = []
Y = []
ppc = 10

HEIGHT =  200
WIDTH = 300

NImagenes = 200
# Recorremos cada una de las carpetas
for folder in os.listdir("raw-img"):
    print("Procesando label: ", folder)
    # Leemos cada una de las imágenes de la carpeta
    for count, file in enumerate(os.listdir(os.path.join("raw-img", folder))[:NImagenes]):
        # Cargamos la imagen y la convertimos a escala de grises
        image = skimage.io.imread(os.path.join("raw-img", folder, file), as_gray=True)

        resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
        # Calculamos las características con HOG
        features = hog(resized_image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2')
        # Añadimos las características y la etiqueta a las listas
        X.append(features)
        Y.append(folder)
        progress = 100 * (count + 1) / len(os.listdir(os.path.join("raw-img", folder))[:NImagenes])
        sys.stdout.write( "\r Procesando Xtest ... (" + str(int(progress)) + " %)")
    print("")

# Separamos el conjunto de datos en entrenamiento y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

print("SHAPE Xtrin",X_train.shape)
print("SHAPE y_train",Y_train.shape)
print("SHAPE X_test",X_test.shape)
print("SHAPE y_test",Y_test.shape)

Procesando label:  perro
 Procesando Xtest ... (100 %)
Procesando label:  caballo
 Procesando Xtest ... (100 %)
Procesando label:  mariposa
 Procesando Xtest ... (51 %)

KeyboardInterrupt: 

In [ ]:
clf = svm.SVC()
hog_features = np.array(hog_features)
data_frame = np.hstack((hog_features,labels))
np.random.shuffle(data_frame)

In [ ]:
#What percentage of data you want to keep for training
percentage = 80
partition = int(len(hog_features)*percentage/100)

In [ ]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()

clf.fit(x_train,y_train)

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))